# Загрузка данных анкет и проведение МАИ, анализ полученных результатов
Данные: заполненные анкеты экспертов (таблицы Excel) 

In [2]:
import re
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
from random import randint

In [3]:

df = pd.read_excel(r'C:\Users\user\матрица\Экс4.xlsx')
df.head()

,Вид происшествия,Аварии,Инциденты,Предпосылки
0,Аварии,1.000000,6.00,8
1,Инциденты,0.166667,1.00,4
2,Предпосылки к инцидентам,0.125000,0.25,1


In [4]:
print('Число строк и столбцов в наборе данных:\n', df.shape)

Число строк и столбцов в наборе данных:
 (3, 4)


In [5]:
# типы столбцов 
df.dtypes

Вид происшествия     object
Аварии              float64
Инциденты           float64
Предпосылки           int64
dtype: object

In [6]:
#удаление столбца типа object для дальнейшей работы с фреймом 
df.pop('Вид происшествия') 
print(df)

     Аварии  Инциденты  Предпосылки
0  1.000000       6.00            8
1  0.166667       1.00            4
2  0.125000       0.25            1


In [7]:
# типы столбцов 
df.dtypes

Аварии         float64
Инциденты      float64
Предпосылки      int64
dtype: object

In [8]:
# также меням тип данных Предпосылки
df['Предпосылки'] = df['Предпосылки'].astype(float)

In [9]:
df.loc[:, df.columns[df.dtypes == 'float64']].head()

,Аварии,Инциденты,Предпосылки
0,1.000000,6.00,8.0
1,0.166667,1.00,4.0
2,0.125000,0.25,1.0


In [10]:
DF_1=df[:].copy()

In [11]:
#Найдем сумму всех столбцов, для того чтобы в дальнейшем пересчитать полученные значения
df.sum ()

Аварии          1.291667
Инциденты       7.250000
Предпосылки    13.000000
dtype: float64

In [12]:
# Считаем значения для нашей матрицы, путем деления каждого значения в столбце на сумму столбца
df['new_Аварии'] = df['Аварии'].astype(float).div(df['Аварии'].sum())
df['new_Инциденты'] = df['Инциденты'].astype(float).div(df['Инциденты'].sum())
df['new_Предпосылки'] = df['Предпосылки'].astype(float).div(df['Предпосылки'].sum())

In [13]:
df.head()

,Аварии,Инциденты,Предпосылки,new_Аварии,new_Инциденты,new_Предпосылки
0,1.000000,6.00,8.0,0.774194,0.827586,0.615385
1,0.166667,1.00,4.0,0.129032,0.137931,0.307692
2,0.125000,0.25,1.0,0.096774,0.034483,0.076923


In [14]:
# Получили новую таблицу теперь отделим значения нашей матрицы от основной таблицы
df_new = df[['new_Аварии', 'new_Инциденты', 'new_Предпосылки']]
df_new

,new_Аварии,new_Инциденты,new_Предпосылки
0,0.774194,0.827586,0.615385
1,0.129032,0.137931,0.307692
2,0.096774,0.034483,0.076923


In [15]:
#Вычислим средние значение по строке и запишем полученный результот в mean
df_new['mean'] = df_new.mean(axis=1)
df_new.head()

C:\Users\user\anaconda3\envs\envpy3.5\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,new_Аварии,new_Инциденты,new_Предпосылки,mean
0,0.774194,0.827586,0.615385,0.739055
1,0.129032,0.137931,0.307692,0.191552
2,0.096774,0.034483,0.076923,0.069393


In [16]:
#проверка, среднее значение в сумме должно быть равно 1
df_new['mean'].sum()

1.0

In [17]:
# Далее проведем проверку на согласованность, 
#для этого создадим из нашей полученной выше таблицы матрицы для дальнейшего их перемножения
df_new4 =df_new['mean']

# удаляем из нашей таблицы значениея столбцы mean
#df_new.pop('mean')

df_new1 =DF_1.loc[0]

df_new2 =DF_1.loc[1]

df_new3 =DF_1.loc[2]

In [18]:
#Проверка на согласованность
res1 = np.dot(df_new1,df_new4)
print(res1)
res2 = np.dot(df_new2,df_new4)
print(res2)
res3 = np.dot(df_new3,df_new4)
print(res3)

2.4435127349476624
0.592301037239858
0.20966315849519407


In [19]:
sum_new = res1 + res2 + res3 
print(sum_new)

3.2454769306827145


In [20]:
# Вычислем значения CI, RI, CR
CI = (sum_new - 3)/(3-1)
print(CI)
RI = (1.98*(3-2))/3
print(RI)
CR = CI/RI
print(CR)

0.12273846534135724
0.66
0.18596737172932915


In [21]:
# Получилось, что значение CR=0.18, что больше 0,1 значит матрицу нельзя считать согласованной
# Проделаем аналогичные действия с другими экспертами